In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('2020_WSB.csv')

In [3]:
# 1ST TEST - credibility
df_response = df[(df.score != 1) & (df.upvote != 1)]
df_response = df_response[(df_response.domain != 'reddit.com')]
# content type (image, video etc.)
df_response['content'] = df_response.domain.apply(lambda x: 'text' if x == 'self.wallstreetbets' 
                      else('image' if x == 'i.redd.it' 
                           else('gif' if x in ['i.imgur.com', 'imgur.com'] 
                                else('video' if x in ['v.redd.it', 'youtu.be', 'youtube.com'] else 'news'))))
df_response['body'][(df_response['body'] == '[removed]') 
                    | (df_response['body'] == '[deleted]') 
                    | (df_response['body'].isnull() == True)] = df_response['content']
df_response['body'][df_response['body'] == 'text'] = ''
df_response['fulltext'] = df_response['title'] + ' ' + df_response['body']
# credit
upvote75 = df_response['upvote'].quantile(0.75)
score75 = df_response['score'].quantile(0.75)
def conditions(s):
    if (s['score'] >= score75) and (s['upvote'] >= upvote75):
        return 'HIGH'
    else:
        return 'LOW'
df_response['credibility'] = df_response.apply(conditions, axis = 1)

<ipython-input-3-799097ac6e58>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_response['body'][(df_response['body'] == '[removed]')
<ipython-input-3-799097ac6e58>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_response['body'][df_response['body'] == 'text'] = ''


In [4]:
df_response

,body,created,domain,id,score,title,upvote,content,fulltext,credibility
4,,2020/01/01 00:32:47,self.wallstreetbets,eif5xu,3.0,Any apps similar to m1?,0.72,text,Any apps similar to m1?,LOW
7,,2020/01/01 01:32:51,self.wallstreetbets,eifobk,0.0,Hot inside tip AMD crashing,0.44,text,Hot inside tip AMD crashing,LOW
9,,2020/01/01 01:51:23,self.wallstreetbets,eiftuc,9.0,Dumbest trades of 2019,0.91,text,Dumbest trades of 2019,LOW
10,,2020/01/01 01:51:58,self.wallstreetbets,eifu11,14.0,DD: AZN / MRK Got Approval For New pancreatic ...,0.79,text,DD: AZN / MRK Got Approval For New pancreatic ...,LOW
11,,2020/01/01 02:03:21,self.wallstreetbets,eifxax,13.0,Happy new decade and shit,0.81,text,Happy new decade and shit,LOW
...,...,...,...,...,...,...,...,...,...,...
299853,image,2020/12/31 23:38:24,i.redd.it,ko5xbz,28.0,Turnip Elon,0.93,image,Turnip Elon image,LOW
299854,image,2020/12/31 23:39:59,i.redd.it,ko5y4i,1767.0,Fr fr,0.99,image,Fr fr image,HIGH
299856,image,2020/12/31 23:45:25,i.redd.it,ko60rl,57.0,"Keep holding bros, we lost this week but WE WI...",0.93,image,"Keep holding bros, we lost this week but WE WI...",HIGH
299858,image,2020/12/31 23:50:52,i.redd.it,ko63fb,12.0,SteveWillDoIt belongs here,0.75,image,SteveWillDoIt belongs here image,LOW


In [283]:
from sklearn.model_selection import train_test_split

In [291]:
X = df_response.loc[:,['fulltext']]
y = df_response.credibility
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y)

In [293]:
X_train_docs = [doc for doc in X_train.fulltext]
X_train_docs

['Theta Gang today, brushing off the sea of red image',
 'Why MGNI is a good long and short term Okay you autists, MGNI just got some new ANAList coverage with a PT of $30. Its currently trading at roughly $20. \n\nMGNI is in the connected tv business and has recent clients such as Disney and hulu. Its closest comparison is TTD which has a market cap 20x that of MGNI. Revenues of the last quarter for MGNI was 60 million, TTD was 216 million. So TTD has roughly 4 times the revenue of MGNI but is worth 20 times more? I think TTD is too high, but MGNI is also way too low and should easily double from here.\n\nI think short term it should get to 28 by February, long term the upside is inverse virgin galactic.',
 "Long on pfizer, MIT has a lab full of gold. The last time I posted here I was given an award then deleted. Fellow autist the depths of my despair from my puts not printing has led me down quite the rabbit hole.\n\nTL;DR from nature it came, from nature their is a solution: Pfizer 